In [ ]:
# define matplotlibplotting backend
# %matplotlib -l shows all available backends
%matplotlib qt

In [ ]:
import os
import numpy as np

from lib.parameters import Parameters
from lib.plot_builder import PlotBuilder
from lib.h5_reader import H5Reader
from lib.plain_reader import PlainReader

In [ ]:
##  configuration options
config_file = '/home/cosmonaut/pdp3_models/model36X71/parameters.xml'
config_file2 = '/home/cosmonaut/pdp3_models/model36X72/parameters.xml'
config_file3 = '/home/cosmonaut/pdp3_models/model36X73/parameters.xml'
config_file4 = '/home/cosmonaut/pdp3_models/model36X81/parameters.xml'

cfg = Parameters(config_file)
cfg2 = Parameters(config_file2)
cfg3 = Parameters(config_file3)
cfg4 = Parameters(config_file4)

radius=0.01
longitude=0.01
time_range=[cfg.start_time, 1e-10]
use_grid=True
ylim=None
use_cache=False
verbose=True

specie = 'electrons'

autoselect = True

x_axis_label = r'$\mathit{t (s)}$'
y_axis_label = r'$\mathit{T (eV)}$'

plot_name = r'$\mathbf{Temperature-time \enspace Dependency}\enspace(T)$'

In [ ]:
# define reader (plain reader used)
if not cfg.use_hdf5:
    reader = PlainReader(path = cfg.data_path,
                         data_root=cfg.data_root,
                         fullframe_size=[cfg.number_r_grid, cfg.number_z_grid],
                         fpds=cfg.frames_per_file,
                         use_cache=use_cache,
                         verbose=verbose)
    reader2 = PlainReader(path = cfg2.data_path,
                          data_root=cfg2.data_root,
                          fullframe_size=[cfg2.number_r_grid, cfg2.number_z_grid],
                          fpds=cfg2.frames_per_file,
                          use_cache=use_cache,
                          verbose=verbose)
    reader3 = PlainReader(path = cfg3.data_path,
                          data_root=cfg3.data_root,
                          fullframe_size=[cfg3.number_r_grid, cfg3.number_z_grid],
                          fpds=cfg3.frames_per_file,
                          use_cache=use_cache,
                          verbose=verbose)
    reader4 = PlainReader(path = cfg4.data_path,
                          data_root=cfg4.data_root,
                          fullframe_size=[cfg4.number_r_grid, cfg4.number_z_grid],
                          fpds=cfg4.frames_per_file,
                          use_cache=use_cache,
                          verbose=verbose)
else:
    reader = H5Reader(str(os.path.join(cfg.data_path, 'data.h5')), use_cache=use_cache)
    reader.verbose = True

In [ ]:
# get data
start_frame = None # cfg.get_frame_number_by_timestamp(time_range[0])
end_frame = None # 460 # cfg.get_frame_number_by_timestamp(time_range[1])
row_number = 100 # cfg.get_row_by_radius(radius)
col_number = 100 # cfg.get_col_by_longitude(longitude)

data = data2 = data3 = data4 = []

for probe in cfg.probes:
    if (probe.type == 'dot') and (probe.specie == specie) and (probe.r_start == row_number) and (probe.z_start == col_number):
        start_frame = cfg.get_frame_number_by_timestamp(time_range[0], probe.schedule)
        end_frame = cfg.get_frame_number_by_timestamp(time_range[1], probe.schedule)
        if probe.component == 'T': 
            data = reader.get_frame_range_dot('T/{}'.format(probe.specie), row_number, col_number, start_frame, end_frame)
            data2 = reader2.get_frame_range_dot('T/{}'.format(probe.specie), row_number, col_number, start_frame, end_frame)
            data3 = reader3.get_frame_range_dot('T/{}'.format(probe.specie), row_number, col_number, start_frame, end_frame)
            data4 = reader4.get_frame_range_dot('T/{}'.format(probe.specie), row_number, col_number, start_frame, end_frame)

if len(data) == 0 and autoselect:
    for probe in cfg.probes:
        if (probe.type == 'col') and (probe.specie == specie) and (probe.z_start == col_number):
            start_frame = cfg.get_frame_number_by_timestamp(time_range[0], probe.schedule)
            end_frame = cfg.get_frame_number_by_timestamp(time_range[1], probe.schedule)
            if probe.component == 'T': 
                data = reader.get_frame_range_col('T/{}'.format(probe.specie), col_number, start_frame, end_frame)[:, row_number]
                data2 = reader2.get_frame_range_col('T/{}'.format(probe.specie), col_number, start_frame, end_frame)[:, row_number]
                data3 = reader3.get_frame_range_col('T/{}'.format(probe.specie), col_number, start_frame, end_frame)[:, row_number]
                data4 = reader4.get_frame_range_col('T/{}'.format(probe.specie), col_number, start_frame, end_frame)[:, row_number]
            break
        elif (probe.type == 'row') and (probe.specie == specie) and (probe.r_start == row_number):
            start_frame = cfg.get_frame_number_by_timestamp(time_range[0], probe.schedule)
            end_frame = cfg.get_frame_number_by_timestamp(time_range[1], probe.schedule)
            if probe.component == 'T':
                data = reader.get_frame_range_row('T/{}'.format(probe.specie), row_number, start_frame, end_frame)[:, col_number]
                data2 = reader2.get_frame_range_row('T/{}'.format(probe.specie), row_number, start_frame, end_frame)[:, col_number]
                data3 = reader3.get_frame_range_row('T/{}'.format(probe.specie), row_number, start_frame, end_frame)[:, col_number]
                data4 = reader4.get_frame_range_row('T/{}'.format(probe.specie), row_number, start_frame, end_frame)[:, col_number]
            break
        elif (probe.type == 'frame') and (probe.specie == specie) and (probe.r_start <= row_number) and (probe.z_start <= col_number) and (probe.r_end >= row_number) and (probe.z_end >= col_number):
            start_frame = cfg.get_frame_number_by_timestamp(time_range[0], probe.schedule)
            end_frame = cfg.get_frame_number_by_timestamp(time_range[1], probe.schedule)
            shape = [probe.r_start, probe.z_start, probe.r_end, probe.z_end]
            if probe.component == 'T': 
                data = reader.get_frame_range('T/{}'.format(probe.specie), shape, start_frame, end_frame)[:, row_number - probe.r_start, col_number - probe.z_start]
                data2 = reader2.get_frame_range('T/{}'.format(probe.specie), shape, start_frame, end_frame)[:, row_number - probe.r_start, col_number - probe.z_start]
                data3 = reader3.get_frame_range('T/{}'.format(probe.specie), shape, start_frame, end_frame)[:, row_number - probe.r_start, col_number - probe.z_start]
                data4 = reader4.get_frame_range('T/{}'.format(probe.specie), shape, start_frame, end_frame)[:, row_number - probe.r_start, col_number - probe.z_start]
data = np.nan_to_num(data)
data2 = np.nan_to_num(data2)
data3 = np.nan_to_num(data3)
data4 = np.nan_to_num(data4)

In [ ]:
data_timeline = np.linspace(time_range[0], time_range[1], len(data))

# define plot builder
plot = PlotBuilder(0, 0, # let the system detects sizes automatically
                   fig_color=cfg.figure_color, 
                   fig_width=cfg.figure_width,
                   fig_height=cfg.figure_height, 
                   fig_dpi=cfg.figure_dpi,
                   font_family=cfg.figure_font_family,
                   font_name=cfg.figure_font_name,
                   font_size=cfg.figure_font_size,
                   tickbox=True, grid=use_grid, is_invert_y_axe=False,
                   aspect='auto', guess_number_ticks=20,
                   # number_x_ticks=10, number_y_ticks=10
                   x_ticklabel_end=1e-9, y_ticklabel_end=1e-9
                  )

# add subplots
plot_obj = plot.add_subplot_cartesian_2d(plot_name, 111, x_axe_label=x_axis_label, y_axe_label=y_axis_label)

# set y-limits
if ylim is not None:
    plot_obj.set_ylim(ylim)

In [ ]:
# add data
plot_obj.plot(data_timeline, data, 'k', label='w1')
plot_obj.plot(data_timeline, data2, 'k--', label='w2')
plot_obj.plot(data_timeline, data3, 'k-.', label='w3')
plot_obj.plot(data_timeline, data4, 'k:', label='w4')


plot_obj.legend(loc='upper left')

plot.show()